In [ ]:
pip install phe

In [ ]:
import time
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
from sympy import symbols, Poly, simplify
import random
import math
from phe import paillier

In [ ]:
random.seed(42)

def generate_random_values(num_parties, values_per_party, num_experiments):
    random_values = {}
    for experiment in range(num_experiments):
        experiment_data = {}
        for party in range(num_parties):
            values = [random.randint(1, 1000) for _ in range(values_per_party)]
            experiment_data[party] = values
        random_values[experiment] = experiment_data
    return random_values

In [ ]:
def no_privacy_protection(num_parties, values_per_party, num_experiments, random_values):
    def generate_data(num_parties, values_per_party, random_values):
        data = {}
        for party in range(num_parties):
            data[party] = random_values[party]  # Use random values from the argument
        return data

    def compute_average(data, num_parties, values_per_party):
        total_sum = 0
        total_count = 0

        for party in range(num_parties):
            total_sum += sum(data[party])
            total_count += len(data[party])

        average = total_sum / total_count
        return average

    average_runtimes = []
    results = []
    accuracy_errors = []

    for experiment in range(num_experiments):

        data = generate_data(num_parties, values_per_party, random_values[experiment])

        start_time = time.time()  # Record start time
        average = compute_average(data, num_parties, values_per_party)
        end_time = time.time()  # Record end time

        runtime = end_time - start_time
        average_runtimes.append(runtime)
        results.append((average, runtime))

        true_average = sum(sum(data[party]) for party in data) / (num_parties * values_per_party)
        accuracy_error = abs(average - true_average)
        accuracy_errors.append(accuracy_error)

    average_runtime = sum(average_runtimes) / num_experiments
    average_accuracy_error = sum(accuracy_errors) / num_experiments


    print(f"Results for n = {num_parties} (No Privacy Protection SMPC):")
    for i, (result, error) in enumerate(zip(results, accuracy_errors)):
        print(f"Experiment {i + 1} - Runtime: {result[1]} seconds")
        print(f"Result: {result[0]}")  # Result is a tuple (average, runtime)
        print(f"Accuracy Error: {error}")
    print(f"Average Runtime for n = {num_parties}: {average_runtime} seconds")
    print(f"Average Accuracy Error for n = {num_parties}: {average_accuracy_error}\n")

    return average_runtime, average_accuracy_error

In [ ]:
def shamir_secret_sharing(num_parties, values_per_party, num_experiments, secret, random_values):
    def share_secret(secret, num_parties, threshold):
        prime_num = 82171  # a large prime number
        coefficients = [secret] + [random.randint(1, prime_num - 1) for _ in range(threshold - 1)]
        x = symbols('x')
        polynomial = sum(coeff * x**i for i, coeff in enumerate(coefficients)) % prime_num
        points = [(i, polynomial.subs(x, i)) for i in range(1, num_parties + 1)]
        return points

    def shamir_reconstruct(points, num_parties):
        x, y = symbols('x y')
        lagrange_basis = 0
        for xi, yi in points:
            basis = yi
            for xj, _ in points:
                if xj != xi:
                    basis *= (x - xj) / (xi - xj)
            lagrange_basis += basis


        # Create a SymPy expression from the computed Lagrange basis
        result = Poly(lagrange_basis, x).as_expr()

        return result

    total_runtime = 0
    results = []
    accuracy_errors = []

    for _ in range(num_experiments):
        start_time = time.time()

        # Sharing the secret
        shares = share_secret(secret, num_parties, num_parties // 2)

        # Combining shares
        result = shamir_reconstruct(shares[:num_parties // 2], num_parties)

        end_time = time.time()
        runtime = end_time - start_time
        total_runtime += runtime

        results.append((result, runtime))

        # Calculate the true secret's value
        true_secret = secret
        # Evaluate the reconstructed result at x = 1 to get the estimated secret
        estimated_secret = result.subs(symbols('x'), 1)
        accuracy_error = abs(estimated_secret - true_secret)
        accuracy_errors.append(accuracy_error)

    average_runtime = total_runtime / num_experiments
    average_accuracy_error = sum(accuracy_errors) / num_experiments

    print(f"Results for n = {num_parties} (Shamir's Secret Sharing SMPC):")
    for i, (result, error) in enumerate(zip(results, accuracy_errors)):
        print(f"Experiment {i + 1} - Runtime: {result[1]} seconds")
        print(f"Result: {result[0]}")
        print(f"Accuracy Error: {error}")
    print(f"Average Runtime for n = {num_parties}: {average_runtime} seconds")
    print(f"Average Accuracy Error for n = {num_parties}: {average_accuracy_error}\n")


    return average_runtime, average_accuracy_error

In [ ]:
def paillier_encryption(num_parties, values_per_experiment, num_experiments, random_values):
    def generate_paillier_keypair():
        public_key, private_key = paillier.generate_paillier_keypair()
        return public_key, private_key

    def paillier_encrypt(public_key, plaintext):
        encrypted_value = public_key.encrypt(plaintext)
        return encrypted_value

    def paillier_decrypt(private_key, ciphertext):
        decrypted_value = private_key.decrypt(ciphertext)
        return decrypted_value


    runtimes = []
    accuracy_errors = []
    experiment_results = []

    for i in range(num_experiments):
        integer_values = random_values[i]

        start_time = time.time()


        public_key, private_key = generate_paillier_keypair()
        encrypted_values = [paillier_encrypt(public_key, val) for val in integer_values]
        sum_encrypted = sum(encrypted_values)
        average_encrypted = paillier_decrypt(private_key, sum_encrypted)

        end_time = time.time()
        runtime = end_time - start_time
        runtimes.append(runtime)

        true_average = sum(integer_values) / len(integer_values)
        accuracy_error = abs(average_encrypted - true_average)
        accuracy_errors.append(accuracy_error)
        experiment_results.append(average_encrypted)

    average_runtime = sum(runtimes) / num_experiments
    average_accuracy_error = sum(accuracy_errors) / num_experiments

    print(f"Results for n = {num_parties} (Paillier Encryption SMPC):")
    for i, (result, runtime, error) in enumerate(zip(experiment_results, runtimes, accuracy_errors)):
        print(f"Experiment {i + 1} - Runtime: {runtime} seconds")
        print(f"Result: {result}")
        print(f"Accuracy Error: {error}")
    print(f"Average Runtime for n = {num_parties}: {average_runtime} seconds")
    print(f"Average Accuracy Error for n = {num_parties}: {average_accuracy_error}\n")

    return average_runtime, average_accuracy_error

In [ ]:
def differential_privacy(num_parties, values_per_party, num_experiments, random_values, epsilon):

    def laplace_mechanism(query_result, epsilon, sensitivity):
        scale = sensitivity / epsilon
        noise = np.random.laplace(0, scale)
        return query_result + noise

    average_runtimes = []
    accuracy_errors = []
    experiment_results = []

    for i in range(num_experiments):
        data = random_values[i]  # Use the existing data

        start_time = time.time()  # Record start time

        total_sum = 0
        total_count = 0

        for party in range(num_parties):
            total_sum += sum(data[party])
            total_count += len(data[party])

        average = total_sum / total_count
        sensitivity = max(data) - min(data)
        noisy_average = laplace_mechanism(average, epsilon, sensitivity)
        experiment_results.append(noisy_average)

        end_time = time.time()  # Record end time
        runtime = end_time - start_time
        average_runtimes.append(runtime)

        true_average = sum(sum(data[party]) for party in data) / (num_parties * values_per_party)
        accuracy_error = abs(noisy_average - true_average)
        accuracy_errors.append(accuracy_error)

    average_runtime = sum(average_runtimes) / num_experiments
    average_accuracy_error = sum(accuracy_errors) / num_experiments

    print(f"Results for n = {num_parties} (Differential Privacy SMPC):")
    for i, (result, runtime, error) in enumerate(zip(experiment_results, average_runtimes, accuracy_errors)):
        print(f"Experiment {i + 1} - Runtime: {runtime} seconds")
        print(f"Result: {result}")
        print(f"Accuracy Error: {error}")
    print(f"Average Runtime for n = {num_parties}: {average_runtime} seconds")
    print(f"Average Accuracy Error for n = {num_parties}: {average_accuracy_error}\n")

    return average_runtime, average_accuracy_error


In [ ]:
values_of_n = [10, 20, 30, 40, 50]

def main():
    num_experiments = 5
    values_per_party = 5

    no_privacy_runtimes = []
    shamir_runtimes = []
    paillier_runtimes = []
    diff_privacy_runtimes = []

    no_privacy_accuracy_errors = []
    shamir_accuracy_errors = []
    paillier_accuracy_errors = []
    diff_privacy_accuracy_errors = []



    for n in values_of_n:
        num_parties = n

        random_values = generate_random_values(num_parties, values_per_party, num_experiments)

        print(f"Running experiments for n = {num_parties} with No Privacy Protection SMPC:")
        runtime, accuracy_error = no_privacy_protection(num_parties, values_per_party, num_experiments, random_values)
        no_privacy_runtimes.append(runtime)
        no_privacy_accuracy_errors.append(accuracy_error)

        print(f"Running experiments for n = {num_parties} with Shamir's Secret Sharing SMPC:")
        secret = 59  # Consider a secret
        runtime, accuracy_error = shamir_secret_sharing(num_parties, values_per_party, num_experiments, secret, random_values)
        shamir_runtimes.append(runtime)
        shamir_accuracy_errors.append(accuracy_error)

        print(f"Running experiments for n = {num_parties} with Paillier Encryption SMPC:")
        runtime, accuracy_error = paillier_encryption(num_parties, values_per_party, num_experiments,random_values)
        paillier_runtimes.append(runtime)
        paillier_accuracy_errors.append(accuracy_error)

        print(f"Running experiments for n = {num_parties} with Differential Privacy SMPC:")
        epsilon = 1.0  # Set the privacy budget
        runtime, accuracy_error = differential_privacy(num_parties, values_per_party, num_experiments, random_values, epsilon)
        diff_privacy_runtimes.append(runtime)
        diff_privacy_accuracy_errors.append(accuracy_error)


    return no_privacy_runtimes, shamir_runtimes, paillier_runtimes, diff_privacy_runtimes, no_privacy_accuracy_errors, shamir_accuracy_errors, paillier_accuracy_errors, diff_privacy_accuracy_errors

if __name__ == "__main__":
    no_privacy_runtimes, shamir_runtimes, paillier_runtimes, diff_privacy_runtimes, no_privacy_accuracy_errors, shamir_accuracy_errors,  paillier_accuracy_errors, diff_privacy_accuracy_errors = main()

    shamir_accuracy_errors = [float(error) for error in shamir_accuracy_errors]

    fig_combined = make_subplots(rows=1, cols=1)
    fig_combined.add_trace(go.Scatter(x=values_of_n, y=no_privacy_runtimes, mode='lines', name='No Privacy'))
    fig_combined.add_trace(go.Scatter(x=values_of_n, y=shamir_runtimes, mode='lines', name="Shamir's Secret Sharing"))
    fig_combined.add_trace(go.Scatter(x=values_of_n, y=paillier_runtimes, mode='lines', name='Paillier Encryption'))
    fig_combined.add_trace(go.Scatter(x=values_of_n, y=diff_privacy_runtimes, mode='lines', name='Differential Privacy'))

    fig_combined.update_layout(
        title='Combined Runtimes for Different SMPC Approaches',
        xaxis_title='n',
        yaxis_title='Runtime (seconds)',)

    fig_combined.show()

    fig_combined_1 = make_subplots(rows=1, cols=1)
    fig_combined_1.add_trace(go.Scatter(x=values_of_n, y=no_privacy_accuracy_errors, mode='lines', name='No Privacy Errors'))
    fig_combined_1.add_trace(go.Scatter(x=values_of_n, y=shamir_accuracy_errors, mode='lines', name="Shamir's Secret Errors"))
    fig_combined_1.add_trace(go.Scatter(x=values_of_n, y=paillier_accuracy_errors, mode='lines', name='Paillier Errors'))
    fig_combined_1.add_trace(go.Scatter(x=values_of_n, y=diff_privacy_accuracy_errors, mode='lines', name='Differential Privacy Errors'))

    fig_combined_1.update_layout(
    title='Combined Accuracy Errors for Different SMPC Approaches',
    xaxis_title='n',
    yaxis_title='Accuracy Error')

    fig_combined_1.show()

